In [20]:
import os 
from dataclasses import dataclass
from typing import List, Tuple
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader

In [21]:
#Loading the PDF file using PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader

def load_pdf(file_path: str):
    loader = PyPDFLoader(file_path)
    pages = loader.load()

    print(f"Total Pages Found: {len(pages)}")
    print("-" * 30)
    print(f"Content of Page 1 (First 500 chars):\n{pages[0].page_content[:500]}")
    print("-" * 30)
    print(f"Metadata of Page 1: {pages[0].metadata}")

    return pages



In [22]:
my_pages=load_pdf('Rag_docs.pdf')

Total Pages Found: 8
------------------------------
Content of Page 1 (First 500 chars):
Candidate:  Gurudevi  Lavanya  Gopisetty  
 
1.  Personal  &  Academic  Overview  
●  Full  Name:  Gurudevi  Lavanya  Gopisetty  ●  Date  of  Birth  (DOB):  june  10  ,  1999  ●  Location:  Long  Beach,  California,  USA  ●  Email:  gglavanya06@gmail.com  ●  Phone:  +1  (669)  306-3851  
Education  
●  Master’s  Degree:  M.S.  in  Computer  Science  ●  University:  California  State  University  ●  Expected  Graduation:  December  2025  ●  Cumulative  GPA:  3.909  /  4.0  
 
2.  Professional  Ro
------------------------------
Metadata of Page 1: {'producer': 'Skia/PDF m145 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Rag_docs', 'source': 'Rag_docs.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}


In [25]:
# creating the chunks to give to the vector database
from langchain_text_splitters import RecursiveCharacterTextSplitter

def create_chunks(text):

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks = splitter.split_documents(text)
    
    return chunks

In [26]:
my_chunks=create_chunks(my_pages)


print(f"✅ Created {len(my_chunks)} chunks.")
print(f"First chunk preview: {my_chunks[0].page_content[:100]}")

✅ Created 20 chunks.
First chunk preview: Candidate:  Gurudevi  Lavanya  Gopisetty  
 
1.  Personal  &  Academic  Overview  
●  Full  Name:  G


In [27]:
# converting the chunks into embbedings into numerical form, so it can understand
from langchain_huggingface import HuggingFaceEmbeddings

def get_embeddings_model():
    model_name= "sentence-transformers/all-MiniLM-L6-v2"
    model_kwargs = {"device": "cpu"}
    encode_kwargs={'normalize_embeddings': False}

    return HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs) 

embed_model= get_embeddings_model()



In [28]:
test_text = "This is a test to see what a vector looks like."
vector = embed_model.embed_query(test_text)

print(f"✅ Embedding Successful!")
print(f"Vector Length: {len(vector)}") # Should be 384 for MiniLM
print(f"First 5 numbers of the vector: {vector[:5]}")

✅ Embedding Successful!
Vector Length: 384
First 5 numbers of the vector: [-0.021258514374494553, 0.03264220803976059, -0.05218735337257385, -0.006472242064774036, 0.06284356117248535]


In [33]:
# now storing the embeddings into a vector database where it can be stored according to its value and positions and can get the similarity by distance calculation
from langchain_chroma import Chroma 
def create_vector_db(chunks, embed_model):
    vector_db = Chroma.from_documents(
        documents=chunks,
        embedding=embed_model,
        persist_directory="./vector_db"
    )
    return vector_db

In [37]:
vector_db = create_vector_db(my_chunks, embed_model)

# --- INSPECTION: The "Similarity Search" Test ---
# Let's see if it can find the right chunk without an LLM
query = "What is the name of the person?" # Change this to a topic in your PDF
search_results = vector_db.similarity_search(query, k=2) # Get top 2 matches

print("\n--- TOP SEARCH RESULT ---")
print(f"Content: {search_results[0].page_content[:200]}...")
print(f"Metadata: {search_results[0].metadata}")


--- TOP SEARCH RESULT ---
Content: Candidate:  Gurudevi  Lavanya  Gopisetty  
 
1.  Personal  &  Academic  Overview  
●  Full  Name:  Gurudevi  Lavanya  Gopisetty  ●  Date  of  Birth  (DOB):  june  10  ,  1999  ●  Location:  Long  Beac...
Metadata: {'producer': 'Skia/PDF m145 Google Docs Renderer', 'title': 'Rag_docs', 'creator': 'PyPDF', 'page': 0, 'source': 'Rag_docs.pdf', 'creationdate': '', 'page_label': '1', 'total_pages': 8}


In [53]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def get_answer(question, vector_db):
    llm = ChatOllama(model="llama3.2:1b")

    # 1. Define the prompt
    template = template = """
    You are a professional and friendly career assistant. 
    Your goal is to answer questions about a candidate's skills based on their resume/profile data.
    your goal is to answer the question as if the candidate answers to the requiterer.

    RULES:
    1. Do NOT say "Based on the context" or "According to the document."
    2. Speak as if you already know the candidate well.
    3. Use a friendly, conversational, and professional tone.
    4. If the information isn't there, politely say you aren't sure about that specific detail.
    5. Keep it concise but enthusiastic.
    6. If you don't have enough information to answer, ask for clarification.
    7. if the question is ask about a job/role she fits in, i want you to answer accordingly , and also put the skills and relevant projects worked on.
    8. start the sentence like  "Lavanya did this, Lavanya did that, Lavanya is a professional in this field, Lavanya has worked on this project, Lavanya is skilled in this area."
    9. if the question is ask about a job/role she fits in, i want you to answer accordingly , and also put the skills and relevant projects worked on.

    Candidate Information:
    {context}
    
    User Question: {question}
    Friendly Response:
   
    """
    prompt = ChatPromptTemplate.from_template(template)

    # 2. Build the chain using the "Pipe" | operator
    # This says: Get context -> Pass to prompt -> Pass to LLM -> Parse as string
    chain = (
        {"context": vector_db.as_retriever(), "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 3. Run it
    return chain.invoke(question)



In [52]:
answer = get_answer("does candidate know machine learning , is she good in it " \
"", vector_db)
print(f"🤖 AI ANSWER:\n{answer}")

🤖 AI ANSWER:
Lavanya has a solid understanding of Machine Learning fundamentals, with strong applied knowledge and mathematical intuition behind models. She's well-versed in the theory-to-practice alignment of ML, including Data Structures & Algorithms, Operating Systems, Finite Automata, Machine Learning, Deep Learning, Artificial Intelligence, Probability, & Statistics.

She also has experience working on projects related to Machine Learning, such as developing predictive models using Python libraries like Scikit-Learn and TensorFlow.
